# Validation of the model with experimental data

We install the specific [galpynostatic package](https://github.com/fernandezfran/galpynostatic) version

In [1]:
%%capture 
!pip install -r requirements.txt

and import the required libraries

In [2]:
import os
import pathlib

import numpy as np
import pandas as pd
import scipy.interpolate

DATA_PATH = pathlib.Path(os.path.join(os.path.abspath("datasets")))
PREDICTIONS_PATH = pathlib.Path(os.path.join(os.path.abspath("predictions")))

In [3]:
import galpynostatic as gp
gp.VERSION

'0.3.3'

We load the experimental data for validation

In [4]:
validation_df = pd.read_csv(DATA_PATH / "validation_data.csv")
validation_df

,Material,d,dcoeff,k0,c1,SOCexp1,c2,SOCexp2,c_rate
0,Graphite,0.000750,1.233000e-10,2.310000e-07,3.0,0.540030,5,0.296843,4.0
1,LTO,0.000175,6.579000e-12,8.100000e-08,2.0,0.654458,5,0.346546,4.0
2,LFP,0.000035,2.800000e-13,1.000000e-09,1.0,0.759624,2,0.329323,1.1
3,LCO,0.002000,5.300000e-09,1.000000e-05,10.0,0.734328,20,0.569661,12.0
4,LMO,0.000003,3.510000e-14,1.900000e-08,20.0,0.863516,50,0.690223,40.0
5,LNMO,0.000800,1.200000e-09,1.000000e-06,12.5,0.725181,25,0.525736,20.0


and make predictions with them

In [5]:
interpolations, predictions = [], []
spherical = pd.read_csv(DATA_PATH / "simulated_map.csv")

for _, sys in validation_df.iterrows():
    flin = scipy.interpolate.interp1d(
        [sys["c1"], sys["c2"]], [sys["SOCexp1"], sys["SOCexp2"]]
    )
    soc_exp = flin(sys["c_rate"])
    interpolations.append(soc_exp)
    
    info = {"d": sys["d"], "dcoeff_": sys["dcoeff"], "k0_": sys["k0"]}
    soc_pred = gp.metric.umbem(
        info, minutes=60.0 / sys["c_rate"], dataset=spherical
    )
    predictions.append(soc_pred)

and we save the predictions

In [6]:
predictions_df = validation_df.copy()
predictions_df["SOC_exp"] = np.asarray(interpolations, dtype=float)
predictions_df["SOC_pred"] = predictions
predictions_df[["Material", "d", "dcoeff", "k0", "SOC_exp", "SOC_pred"]]

,Material,d,dcoeff,k0,SOC_exp,SOC_pred
0,Graphite,0.000750,1.233000e-10,2.310000e-07,0.418436,0.445421
1,LTO,0.000175,6.579000e-12,8.100000e-08,0.449183,0.444614
2,LFP,0.000035,2.800000e-13,1.000000e-09,0.716593,0.778499
3,LCO,0.002000,5.300000e-09,1.000000e-05,0.701395,0.716463
4,LMO,0.000003,3.510000e-14,1.900000e-08,0.747987,0.770205
5,LNMO,0.000800,1.200000e-09,1.000000e-06,0.605514,0.641796


In [7]:
predictions_df[["Material", "d", "dcoeff", "k0", "SOC_exp", "SOC_pred"]].to_csv(
    PREDICTIONS_PATH / "validation_charging_rate.csv", index=False
)